In [ ]:
!pip -q install librosa==0.10.2.post1 soundfile==0.12.1 scikit-learn==1.6.1 matplotlib==3.9.2
# TF is preinstalled on Colab; if local:
# !pip -q install tensorflow==2.15.0


In [ ]:
import os, random, json, math, hashlib, tarfile, shutil, warnings
from pathlib import Path
import numpy as np
import pandas as pd
import librosa, soundfile as sf
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, f1_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# --- Reproducibility ---
SEED = 1337
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)

# --- Choose comparison mode ---
USE_SIX_CLASSES = True   # True → 6-class subset (your classes); False → all 10 classes

# --- Dataset paths ---
CFG = {
    "root": "/content/UrbanSound8K",
    "metadata_csv": "metadata/UrbanSound8K.csv",
    "folds_dir": "audio",
    "cache_dir": "/content/us8k_cache_mfcc",
    "out_dir": "/content/outputs_hassan_baseline"
}
Path(CFG["cache_dir"]).mkdir(parents=True, exist_ok=True)
Path(CFG["out_dir"]).mkdir(parents=True, exist_ok=True)

# --- Audio/feature params (Hassan-style MFCC) ---
SR = 22050
CLIP_SECONDS = 4.0
N_MFCC = 40
N_FFT = 2048
HOP = 512
FIXED_FRAMES = 174   # pad/truncate to this many frames, common in MFCC examples
MONO = True

# --- Class list ---
CLASSES_6 = ["car_horn","engine_idling","siren","dog_bark","drilling","street_music"]
# For 10-class: we’ll derive from metadata later

meta_path = Path(CFG["root"]) / CFG["metadata_csv"]
print("Using 6-class mode:", USE_SIX_CLASSES)


Using 6-class mode: True


In [ ]:
if not meta_path.exists():
    dl_dir = Path("/content/_dl"); dl_dir.mkdir(parents=True, exist_ok=True)
    archive = dl_dir / "UrbanSound8K.tar.gz"
    if not archive.exists():
        url1 = "https://zenodo.org/records/1203745/files/UrbanSound8K.tar.gz?download=1"
        url2 = "https://urbansounddataset.weebly.com/uploads/5/3/6/2/5362550/urbansound8k.tar.gz"
        print("Downloading UrbanSound8K…")
        rc = os.system(f'wget -q --show-progress -O "{archive}" "{url1}"')
        if rc != 0 or not archive.exists() or archive.stat().st_size < 100_000_000:
            print("Zenodo failed; trying mirror…")
            os.system(f'wget -q --show-progress -O "{archive}" "{url2}"')
    print("Extracting…")
    with tarfile.open(archive, "r:gz") as tar:
        tar.extractall("/content")
    # Normalize to /content/UrbanSound8K
    candidates = list(Path("/content").glob("**/UrbanSound8K/metadata/UrbanSound8K.csv"))
    assert candidates, "metadata not found after extraction"
    found_root = candidates[0].parents[1]
    if str(found_root) != str(Path(CFG["root"])):
        if Path(CFG["root"]).exists(): shutil.rmtree(CFG["root"])
        shutil.move(str(found_root), CFG["root"])

print("✅ Dataset ready:", meta_path.exists(), meta_path)


✅ Dataset ready: True /content/UrbanSound8K/metadata/UrbanSound8K.csv


In [ ]:
meta = pd.read_csv(meta_path)
assert {"fold","slice_file_name","class"}.issubset(meta.columns)

# Build full path
meta["fname"] = meta["slice_file_name"]
meta["fold_path"] = meta["fold"].apply(lambda f: str(Path(CFG["root"]) / CFG["folds_dir"] / f"fold{int(f)}"))
meta["path"] = meta.apply(lambda r: str(Path(r["fold_path"]) / r["fname"]), axis=1)

if USE_SIX_CLASSES:
    meta = meta[meta["class"].isin(CLASSES_6)].reset_index(drop=True)
    classes = sorted(CLASSES_6)
else:
    classes = sorted(meta["class"].unique().tolist())

label2id = {c:i for i,c in enumerate(classes)}
id2label = {v:k for k,v in label2id.items()}
meta["target"] = meta["class"].map(label2id)

# Folds: train=1–8, val=9, test=10
train_df = meta[meta["fold"].isin(list(range(1,9)))].reset_index(drop=True)
val_df   = meta[meta["fold"]==9].reset_index(drop=True)
test_df  = meta[meta["fold"]==10].reset_index(drop=True)

print("Classes:", classes)
print("train/val/test sizes:", len(train_df), len(val_df), len(test_df))
print("Example file exists:", Path(train_df.loc[0,"path"]).exists())


Classes: ['car_horn', 'dog_bark', 'drilling', 'engine_idling', 'siren', 'street_music']
train/val/test sizes: 4346 503 509
Example file exists: True


In [ ]:
from functools import lru_cache

Path(CFG["cache_dir"]).mkdir(parents=True, exist_ok=True)
clip_len = int(SR * CLIP_SECONDS)

def _cache_key(wav_path: str) -> Path:
    h = hashlib.sha1(wav_path.encode()).hexdigest()[:16]
    return Path(CFG["cache_dir"]) / f"{h}_mfcc{N_MFCC}_hop{HOP}_frames{FIXED_FRAMES}.npy"

def load_clip_fixed(path: str):
    y, _ = librosa.load(path, sr=SR, mono=MONO)
    if len(y) < clip_len:
        y = np.pad(y, (0, clip_len - len(y)))
    else:
        y = y[:clip_len]
    return y

def wav_to_mfcc_fixed(y: np.ndarray):
    mfcc = librosa.feature.mfcc(y=y, sr=SR, n_mfcc=N_MFCC, n_fft=N_FFT, hop_length=HOP)
    # Normalize per-utterance (mean-variance)
    mu, sigma = mfcc.mean(axis=1, keepdims=True), mfcc.std(axis=1, keepdims=True) + 1e-6
    mfcc = (mfcc - mu)/sigma
    # Pad/trim to FIXED_FRAMES along time axis
    T = mfcc.shape[1]
    if T < FIXED_FRAMES:
        mfcc = np.pad(mfcc, ((0,0),(0, FIXED_FRAMES - T)))
    else:
        mfcc = mfcc[:, :FIXED_FRAMES]
    # Add channel dim → (H, W, 1)
    return mfcc[..., None].astype(np.float32)

def mfcc_from_path(wav_path: str):
    ck = _cache_key(wav_path)
    if ck.exists():
        return np.load(ck)
    y = load_clip_fixed(wav_path)
    x = wav_to_mfcc_fixed(y)
    np.save(ck, x)
    return x


In [ ]:
def build_xy(df: pd.DataFrame):
    X = np.stack([mfcc_from_path(p) for p in df["path"].tolist()], axis=0)  # (N, 40, 174, 1)
    y = df["target"].to_numpy().astype(np.int32)
    Y = keras.utils.to_categorical(y, num_classes=len(classes)).astype(np.float32)
    return X, Y

print("Building arrays (this will cache MFCCs to disk on first run)…")
X_train, Y_train = build_xy(train_df)
X_val,   Y_val   = build_xy(val_df)
X_test,  Y_test  = build_xy(test_df)
X_train.shape, Y_train.shape


Building arrays (this will cache MFCCs to disk on first run)…


((4346, 40, 174, 1), (4346, 6))

In [ ]:
def make_hassan_cnn(input_shape=(N_MFCC, FIXED_FRAMES, 1), n_classes=10):
    inp = keras.Input(shape=input_shape)
    x = layers.Conv2D(16, (3,3), padding='same', activation='relu')(inp)
    x = layers.MaxPool2D((2,2))(x); x = layers.Dropout(0.2)(x)

    x = layers.Conv2D(32, (3,3), padding='same', activation='relu')(x)
    x = layers.MaxPool2D((2,2))(x); x = layers.Dropout(0.2)(x)

    x = layers.Conv2D(64, (3,3), padding='same', activation='relu')(x)
    x = layers.MaxPool2D((2,2))(x); x = layers.Dropout(0.2)(x)

    x = layers.Conv2D(128,(3,3), padding='same', activation='relu')(x)
    x = layers.MaxPool2D((2,2))(x); x = layers.Dropout(0.2)(x)

    x = layers.GlobalAveragePooling2D()(x)
    out = layers.Dense(n_classes, activation='softmax')(x)
    m = keras.Model(inp, out)
    m.compile(optimizer=keras.optimizers.Adam(learning_rate=2e-3),
              loss='categorical_crossentropy',
              metrics=[keras.metrics.CategoricalAccuracy(name='acc')])
    return m

model = make_hassan_cnn(n_classes=len(classes))
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 40, 174, 1)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 40, 174, 16)    │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 20, 87, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 20, 87, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 20, 87, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 10, 43, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 10, 43, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 10, 43, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 5, 21, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 5, 21, 64)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 5, 21, 128)     │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 2, 10, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 2, 10, 128)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 97,926 (382.52 KB)

 Trainable params: 97,926 (382.52 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# ⬅️ replace your MacroF1Checkpoint class + its instantiation with this

from sklearn.metrics import f1_score
from tensorflow import keras
from pathlib import Path
import numpy as np

# choose a compliant filename (Keras 3 requires ".weights.h5")
WEIGHTS_PATH = Path(CFG["out_dir"]) / "best_hassan.weights.h5"

class MacroF1Checkpoint(keras.callbacks.Callback):
    def __init__(self, x_val, y_val, path: Path):
        super().__init__()
        self.x_val, self.y_val = x_val, y_val
        # enforce the required suffix
        self.path = Path(path)
        if not str(self.path).endswith(".weights.h5"):
            self.path = self.path.with_suffix(".weights.h5")
        self.best = -1.0
        self.history_f1 = []

    def on_epoch_end(self, epoch, logs=None):
        y_pred = self.model.predict(self.x_val, verbose=0)
        y_true = np.argmax(self.y_val, axis=1)
        y_hat  = np.argmax(y_pred, axis=1)
        f1 = f1_score(y_true, y_hat, average='macro')
        self.history_f1.append(float(f1))
        if f1 > self.best:
            self.best = f1
            self.model.save_weights(self.path)  # ✅ now ends with .weights.h5
        print(f"\n[MacroF1] val_macroF1={f1:.4f} (best={self.best:.4f})")

f1_ckpt = MacroF1Checkpoint(X_val, Y_val, WEIGHTS_PATH)

# keep EarlyStopping as is (monitoring val_accuracy)
es = keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=6, restore_best_weights=False)



In [ ]:
history = model.fit(
    X_train, Y_train,
    validation_data=(X_val, Y_val),
    epochs=30,
    batch_size=64,
    callbacks=[f1_ckpt, es],
    verbose=1
)

# Reload best by macro-F1
model.load_weights(WEIGHTS_PATH)  # ✅ points to best_hassan.weights.h5



Epoch 1/30
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 730ms/step - acc: 0.2543 - loss: 1.7456
[MacroF1] val_macroF1=0.4117 (best=0.4117)
68/68 ━━━━━━━━━━━━━━━━━━━━ 56s 766ms/step - acc: 0.2555 - loss: 1.7435 - val_acc: 0.5010 - val_loss: 1.3984
Epoch 2/30


/usr/local/lib/python3.12/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_accuracy` which is not available. Available metrics are: acc,loss,val_acc,val_loss
  current = self.get_monitor_value(logs)


68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 473ms/step - acc: 0.5290 - loss: 1.2788
[MacroF1] val_macroF1=0.5718 (best=0.5718)
68/68 ━━━━━━━━━━━━━━━━━━━━ 64s 501ms/step - acc: 0.5294 - loss: 1.2782 - val_acc: 0.6461 - val_loss: 1.0603
Epoch 3/30


/usr/local/lib/python3.12/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_accuracy` which is not available. Available metrics are: acc,loss,val_acc,val_loss
  current = self.get_monitor_value(logs)


68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 498ms/step - acc: 0.6341 - loss: 1.0305
[MacroF1] val_macroF1=0.7389 (best=0.7389)
68/68 ━━━━━━━━━━━━━━━━━━━━ 36s 532ms/step - acc: 0.6341 - loss: 1.0305 - val_acc: 0.7495 - val_loss: 0.8909
Epoch 4/30


...

/usr/local/lib/python3.12/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_accuracy` which is not available. Available metrics are: acc,loss,val_acc,val_loss
  current = self.get_monitor_value(logs)


68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 474ms/step - acc: 0.9041 - loss: 0.2754
[MacroF1] val_macroF1=0.8284 (best=0.8482)
68/68 ━━━━━━━━━━━━━━━━━━━━ 71s 501ms/step - acc: 0.9040 - loss: 0.2755 - val_acc: 0.8469 - val_loss: 0.4669
Epoch 30/30


/usr/local/lib/python3.12/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_accuracy` which is not available. Available metrics are: acc,loss,val_acc,val_loss
  current = self.get_monitor_value(logs)


68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 494ms/step - acc: 0.9031 - loss: 0.2555
[MacroF1] val_macroF1=0.8462 (best=0.8482)
68/68 ━━━━━━━━━━━━━━━━━━━━ 43s 537ms/step - acc: 0.9031 - loss: 0.2555 - val_acc: 0.8588 - val_loss: 0.4512


/usr/local/lib/python3.12/dist-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_accuracy` which is not available. Available metrics are: acc,loss,val_acc,val_loss
  current = self.get_monitor_value(logs)


In [ ]:
# Predict test
y_true = np.argmax(Y_test, axis=1)
y_prob = model.predict(X_test, batch_size=128, verbose=0)
y_pred = np.argmax(y_prob, axis=1)

# Metrics
rep = classification_report(y_true, y_pred, target_names=classes, output_dict=True)
acc = float((y_true==y_pred).mean())
macro_f1 = float(rep["macro avg"]["f1-score"])
print("TEST  acc=", acc, " macro-F1=", macro_f1)

# Save metrics
metrics = {"test_acc": acc, "test_macro_f1": macro_f1, "test_report": rep,
           "mode": "6-classes" if USE_SIX_CLASSES else "10-classes"}
with open(Path(CFG["out_dir"])/"metrics_hassan.json","w") as f:
    json.dump(metrics, f, indent=2)

# Confusion
cm = confusion_matrix(y_true, y_pred)
cm_df = pd.DataFrame(cm, index=classes, columns=classes)
cm_df.to_csv(Path(CFG["out_dir"]) / "confusion_hassan.csv", index=True)

# Plot confusion
fig, ax = plt.subplots(figsize=(6,5))
im = ax.imshow(cm, interpolation='nearest')
ax.set_title("Confusion Matrix — Hassan MFCC+CNN"); ax.set_xlabel("Predicted"); ax.set_ylabel("True")
ax.set_xticks(range(len(classes))); ax.set_yticks(range(len(classes)))
ax.set_xticklabels(classes, rotation=45, ha='right'); ax.set_yticklabels(classes)
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(j, i, cm[i, j], ha="center", va="center", fontsize=9)
plt.tight_layout(); fig.savefig(Path(CFG["out_dir"])/"confusion_hassan.png", dpi=170); plt.close(fig)

print("Saved metrics/confusion to:", CFG["out_dir"])


TEST  acc= 0.787819253438114  macro-F1= 0.7568832982202179
Saved metrics/confusion to: /content/outputs_hassan_baseline


In [ ]:
# Per-class table (CSV + Markdown)
per_rows=[]
for cls in classes:
    s = rep[cls]
    per_rows.append([cls, s['precision'], s['recall'], s['f1-score'], s['support']])
per_df = pd.DataFrame(per_rows, columns=['Class','Precision','Recall','F1-score','Support'])
per_df.to_csv(Path(CFG["out_dir"])/"per_class_hassan.csv", index=False)

md = ["| Class | Precision | Recall | F1-score | Support |\n", "|---|---:|---:|---:|---:|\n"]
for _,r in per_df.iterrows():
    md.append(f"| {r['Class']} | {r['Precision']:.3f} | {r['Recall']:.3f} | {r['F1-score']:.3f} | {int(r['Support'])} |\n")
md.append(f"| **Macro-F1 (overall)** |  |  | **{macro_f1:.3f}** | **{int(per_df['Support'].sum())}** |\n")
(Path(CFG["out_dir"])/"per_class_hassan.md").write_text("".join(md), encoding="utf-8")

# Training curves
h = history.history
fig1, ax1 = plt.subplots(figsize=(6,4))
ax1.plot(h['loss'], label='train_loss'); ax1.plot(h['val_loss'], label='val_loss')
ax1.set_title('Loss'); ax1.legend(); plt.tight_layout(); fig1.savefig(Path(CFG["out_dir"])/"curve_loss_hassan.png", dpi=170); plt.close(fig1)

fig2, ax2 = plt.subplots(figsize=(6,4))
ax2.plot(h['acc'], label='train_acc'); ax2.plot(h['val_acc'], label='val_acc')
ax2.set_title('Accuracy'); ax2.legend(); plt.tight_layout(); fig2.savefig(Path(CFG["out_dir"])/"curve_acc_hassan.png", dpi=170); plt.close(fig2)

fig3, ax3 = plt.subplots(figsize=(6,4))
ax3.plot(f1_ckpt.history_f1, label='val_macroF1 (per epoch)')
ax3.set_title('Val Macro-F1 (by callback)'); ax3.legend(); plt.tight_layout()
fig3.savefig(Path(CFG["out_dir"])/"curve_val_macroF1_hassan.png", dpi=170); plt.close(fig3)

print("Saved per-class table + curves to:", CFG["out_dir"])


Saved per-class table + curves to: /content/outputs_hassan_baseline


In [ ]:
cmp_lines = [
"| Model | Classes | Test Acc | Test Macro-F1 |\n",
"|---|---:|---:|---:|\n",
f"| Hassan MFCC+CNN (reproduced) | {len(classes)} | {acc:.3f} | {macro_f1:.3f} |\n",
]
(Path(CFG["out_dir"])/"comparison_snippet_hassan.md").write_text("".join(cmp_lines), encoding="utf-8")
print("Saved:", Path(CFG["out_dir"])/"comparison_snippet_hassan.md")


Saved: /content/outputs_hassan_baseline/comparison_snippet_hassan.md


In [ ]:
from pathlib import Path
import shutil, os

HASSAN_DIR = Path("/content/outputs_hassan_baseline")  # <- change if you saved elsewhere
assert HASSAN_DIR.exists(), f"Folder not found: {HASSAN_DIR}"

zip_path = "/content/hassan_baseline_artifacts.zip"
# create/overwrite the zip
shutil.make_archive(zip_path.replace(".zip",""), "zip", str(HASSAN_DIR))
print("Zipped to:", zip_path)


Zipped to: /content/hassan_baseline_artifacts.zip
